In [6]:
import requests
import pandas as pd
import time

KAKAO_API_KEY = "51238f851939f847bef643cdbe2bd318"

def get_lat_lon(address):
    if pd.isna(address) or address.strip() == "":
        return None, None  # 빈 값 처리

    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {"query": address}

    response = requests.get(url, headers=headers, params=params)
    result = response.json()

    # time.sleep(0.2)  # API 호출 속도 조절

    if result.get("documents"):
        lat = result["documents"][0]["y"]
        lon = result["documents"][0]["x"]
        print(f"✅ 변환 성공: {address} → ({lat}, {lon})")
        return lat, lon
    else:
        print(f"❌ 변환 실패: {address}")
        return None, None

# CSV 파일 불러오기
file_path = "충청북도 청주시_음식점 현황_20240821.csv"

try:
    df = pd.read_csv(file_path, encoding="cp949")
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding="euc-kr")

print("📂 파일 로드 완료")
print(df.head())  # 데이터 미리보기

# D열(4번째 열)의 주소 데이터 가져오기
address_column = df.columns[3]  # D열의 정확한 컬럼명 가져오기
df["위도"], df["경도"] = zip(*df[address_column].apply(get_lat_lon))  # 원본 df 크기 유지

# 변환된 데이터를 새로운 CSV 파일로 저장
output_file = "주소_좌표변환_완료.csv"
df.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"🎉 변환 완료! 저장된 파일: {output_file}")


📂 파일 로드 완료
     업종명      업소명                               소재지(도로명)  \
0  일반음식점      부산집                                    NaN   
1  일반음식점     청송통닭           충청북도 청주시 상당구 남사로 116 (남문로2가)   
2  일반음식점      공원당      충청북도 청주시 상당구 상당로55번길 40-2 (남문로2가)   
3  일반음식점      함바구  충청북도 청주시 상당구 상당로131번길 7-2, 1층 (북문로2가)   
4  일반음식점  왕천파닭중앙점            충청북도 청주시 상당구 중앙로 53 (북문로3가)   

                      소재지(지번)         소재지전화     업태명  
0     충청북도 청주시 상당구 남문로1가 243            NaN      한식  
1   충청북도 청주시 상당구 남문로2가 113-6   043-255-6536      한식  
2      충청북도 청주시 상당구 남문로2가 87   043-255-3894      분식  
3  충청북도 청주시 상당구 북문로2가 113-25   043-257-6776     경양식  
4    충청북도 청주시 상당구 북문로3가 87-6   043-223-0976  통닭(치킨)  
❌ 변환 실패: 충청북도 청주시 상당구 남문로1가 243 
✅ 변환 성공: 충청북도 청주시 상당구 남문로2가 113-6  → (36.631847350228, 127.48782959381)
✅ 변환 성공: 충청북도 청주시 상당구 남문로2가 87  → (36.6330884526891, 127.488604648462)
✅ 변환 성공: 충청북도 청주시 상당구 북문로2가 113-25  → (36.6399824962565, 127.489316187269)
✅ 변환 성공: 충청북도 청주시 상당구 북문로3가 87-6  → (36.6420091092635

In [36]:
import folium

file_path = "주소_좌표변환_완료.csv"

# CSV 파일 읽기
df = pd.read_csv(file_path, encoding='utf-8')

# G, H 열에서 널값이 있는 행 제거
df = df.dropna(subset=['업소명','위도', '경도'])

# 위도와 경도 정보
latitude = df['위도']
longitude = df['경도']
restaurant = df['업소명']
# 지도 초기화 (기본 위치는 첫 번째 좌표로 설정)
map_center = [36.60672, 127.4844]
my_map = folium.Map(location=map_center, zoom_start= 20)

# 각 좌표에 마커 추가
for name, lat, lon in zip(restaurant, latitude, longitude):
    folium.Marker([lat, lon],popup =f"<pre style='white-space: pre-wrap; width: 100px;'> {name}</pre>").add_to(my_map)

# 지도 저장 (HTML 파일로 저장)
my_map.save("map_output.html")

In [39]:
import folium
from folium.plugins import MarkerCluster, HeatMap

file_path = "주소_좌표변환_완료.csv"

# CSV 파일 읽기
df = pd.read_csv(file_path, encoding='utf-8')


df = df.dropna(subset=['업소명', '위도', '경도'])

# 위도와 경도 정보
latitude = df['위도']
longitude = df['경도']
restaurant = df['업소명']

# 지도 초기화 (기본 위치는 첫 번째 좌표로 설정)
map_center = [36.60672, 127.4844]
my_map = folium.Map(location=map_center, zoom_start=12)

# MarkerCluster 레이어 생성
marker_cluster_layer = folium.FeatureGroup(name="Marker Clusters").add_to(my_map)
marker_cluster = MarkerCluster().add_to(marker_cluster_layer)
for name, lat, lon in zip(restaurant, latitude, longitude):
    folium.Marker(
        location=[lat, lon],
        popup=f"<pre style='white-space: pre-wrap; width: 100px;'> {name}</pre>"
    ).add_to(marker_cluster)

# 마커 레이어 생성
marker_layer = folium.FeatureGroup(name="Markers").add_to(my_map)
for name, lat, lon in zip(restaurant, latitude, longitude):
    folium.Marker(
        location=[lat, lon],
        popup=f"<pre style='white-space: pre-wrap; width: 100px;'> {name}</pre>"
    ).add_to(marker_layer)

# 원형 마커 레이어 생성
circle_layer = folium.FeatureGroup(name="Circle Markers").add_to(my_map)
for name, lat, lon in zip(restaurant, latitude, longitude):
    folium.CircleMarker(
        location=[lat, lon], 
        radius=10, 
        color='blue', 
        fill=True, 
        fill_color='blue', 
        fill_opacity=0.6,
        popup=f"<pre style='white-space: pre-wrap; width: 100px;'> {name}</pre>"
    ).add_to(circle_layer)

# Heatmap 레이어 생성
heatmap_layer = folium.FeatureGroup(name="Heatmap").add_to(my_map)
heat_data = [[lat, lon] for lat, lon in zip(latitude, longitude)]
HeatMap(heat_data).add_to(heatmap_layer)

# LayerControl을 추가하여 사용자가 레이어를 선택/해제할 수 있도록 함
folium.LayerControl().add_to(my_map)

# 지도 저장 (HTML 파일로 저장)
my_map.save("map_with_layer_control.html")


In [48]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd

file_path = "주소_좌표변환_완료.csv"

# CSV 파일 읽기
df = pd.read_csv(file_path, encoding='utf-8')

# 널값 제거
df = df.dropna(subset=['업소명', '위도', '경도'])

# 위도와 경도 정보
latitude = df['위도']
longitude = df['경도']
restaurant = df['업소명']

# 지도 초기화 (기본 위치는 첫 번째 좌표로 설정)
map_center = [36.60672, 127.4844]
my_map = folium.Map(location=map_center, zoom_start=12)

# MarkerCluster 레이어 생성
marker_cluster_layer = folium.FeatureGroup(name="Marker Clusters").add_to(my_map)
marker_cluster = MarkerCluster().add_to(marker_cluster_layer)

# 각 마커 추가
for name, lat, lon in zip(restaurant, latitude, longitude):
    folium.Marker(
        location=[lat, lon],
        popup=f"<pre style='white-space: pre-wrap; width: 100px;'> {name}</pre>"
    ).add_to(marker_cluster)

# 지도 탭 제목 설정
my_map.get_root().html.add_child(folium.Element("<title>청주 음식점 지도</title>"))

# 지도 저장 (HTML 파일로 저장)
my_map.save("청주음식점지도.html")
